<a href="https://colab.research.google.com/github/Marshmellowon/DeepLearning_with_Pytorch/blob/master/Mnist_CNN_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch 
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

import torch.nn.init

In [ ]:
# GPU 설정

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

print(device)

In [ ]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [153]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [154]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True, # 데이터를 섞은 상태로 던져줘라
                                          drop_last=True)

In [155]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer1 = nn.Sequential(nn.Conv2d(1,32, kernel_size=3, stride=1, padding=1),
                                nn.ReLU(),
                                nn.MaxPool2d(2))
    
    self.layer2 = nn.Sequential(nn.Conv2d(32,64,kernel_size=3, stride=1, padding=1),
                                nn.ReLU(),
                                nn.MaxPool2d(2))
    self.layer3 = nn.Sequential(nn.Conv2d(64,128,kernel_size=3, stride=1, padding=1),
                                nn.ReLU(),
                                nn.MaxPool2d(2))
    
    # fooly connected layer
    self.fc1 = nn.Linear(3*3*128, 625)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(625, 10, bias=True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    torch.nn.init.xavier_uniform_(self.fc2.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    
    out = out.view(out.size(0),-1)
    out = self.fc1(out)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [156]:
model = CNN().to(device)
print(model)

value = torch.Tensor(1,1,28,28,).to(device)
print((model(value)).shape)   

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1152, out_features=625, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=625, out_features=10, bias=True)
)
torch.Size([1, 10])


In [157]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [158]:
# training
total_batch = len(data_loader)
print('Learning Start')

for epoch in range(training_epochs):
  avg_cost = 0

  for X,Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad() # 매우 중요 빼먹으면 학습 안됨
    hypothesis = model(X)

    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost/total_batch

  print('[Epoch: {}] cost = {}'.format(epoch+1, avg_cost))

print('Learning fin!!')

Learning Start
[Epoch: 1] cost = 0.16031615436077118
[Epoch: 2] cost = 0.04234233871102333
[Epoch: 3] cost = 0.03109152801334858
[Epoch: 4] cost = 0.02314816601574421
[Epoch: 5] cost = 0.01905001513659954
[Epoch: 6] cost = 0.014574842527508736
[Epoch: 7] cost = 0.011850343085825443
[Epoch: 8] cost = 0.012958429753780365
[Epoch: 9] cost = 0.008543276228010654
[Epoch: 10] cost = 0.009873151779174805
[Epoch: 11] cost = 0.006704459898173809
[Epoch: 12] cost = 0.007122073322534561
[Epoch: 13] cost = 0.006904544774442911
[Epoch: 14] cost = 0.007137093227356672
[Epoch: 15] cost = 0.004373317118734121
Learning fin!!


In [159]:
with torch.no_grad():
  X_test = mnist_test.test_data.view(len(mnist_test), 1,28,28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction,1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy: ', accuracy.item())

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy:  0.9901000261306763


- 레이어를 깊게 쌓으면 어떻게 될까??
- 레이어를 깊게 쌓는다고 무조건 정확도가 높아지지는 않는다.